# Actual Code to remove hair and put into file

In [12]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import threading
# import sys

# import time, random
# from atpbar import atpbar

path = '../../removed_hair_images2/'

# try:
#     os.mkdir(path)
# except OSError:
#     print ("Creation of the directory %s failed" % path)
# else:
#     print ("Successfully created the directory %s " % path)

imageNames = [
    "5436e3c6bae478396759f233.jpg",
    "5aaf1253116597691362835b.jpg",
    "5aaf12ba116597691362b671.jpg",
    "5aaf13841165976913631d42.jpg",
    "5aaf1320116597691362ea91.jpg",
    "5592cfca9fc3c131551458e7.jpg",
    "5436e3b8bae478396759f18d.jpg"
]
numberOfProcessedImages = 0

In [13]:
import os
import glob
img_dir = "C:/Users/ava/Desktop/KSU/Year 4/Spring 2020/Senior Project/NewCroppedImages/NewCroppedImages/" # Enter Directory of all images 
data_path = os.path.join(img_dir,'*g')
files = glob.glob(data_path)
imageNames = []

for f1 in files:
#     img = cv2.imread(f1)
#     name = f1[-28: -4]
    imageNames.append(f1)
    
print(len(imageNames))
print(imageNames[0])

1210
C:/Users/ava/Desktop/KSU/Year 4/Spring 2020/Senior Project/NewCroppedImages/NewCroppedImages\5436e3abbae478396759f0cf.jpg


In [14]:

lck = threading.Lock()

def writeToFile(index, img):
    global lck
    lck.acquire()
    global numberOfProcessedImages
    cv2.imwrite(path + imageNames[index][-28:], img)
    numberOfProcessedImages = numberOfProcessedImages + 1
    print(str("Processed: " + str(numberOfProcessedImages) + "/" + str(len(imageNames)) + " ==> " + str(round((numberOfProcessedImages / len(imageNames)), 3) * 100) + "%"),
          end="                         \r", flush=True)
    lck.release()

In [15]:
def zipCoordinates(edges):
    indices = np.where(edges >= [40])
    coordinates = zip(indices[0], indices[1])
    return coordinates

def getBlackHighlights(img, edgeCoordinates):
    for c in edgeCoordinates:
        img[c[0], c[1]] = [0,0,0]
    
    return img

In [16]:
def getCoordinates(index, radius, w, h):
#     startingCorner = [index[0] - radius, index[1] - radius]
    result = []
    for i in range(index[0] - radius, index[0] + radius + 1):
        if i >= h:
            break
        if i >= 0:
            for j in range(index[1] - radius, index[1] + radius + 1):
                if j >= w:
                    break
                if j >= 0:
                    result.append([i, j])

            
#     print(result)
    return result

In [17]:
from statistics import mean 
from random import randint

def blurHairs2(img, edges, imageWithBlack):
    indices = np.where(edges >= [40])
    coordinates = zip(indices[0], indices[1])
    
    img2 =  img      
    h, w, c = img2.shape
#     print(h, w, c)

    for c in coordinates:
        x = c[0]
        y = c[1]
        r = 0
        g = 0
        b = 0
        count = 0
        exceptions = 0
        tempCoordinates = getCoordinates([x,y], randint(4, 15), w, h)#[[x, y+1], [x, y-1], [x+1, y], [x+1, y+1], [x+1, y-1], [x-1, y], [x-1, y+1], [x-1, y-1]]
        for c2 in tempCoordinates:
            x2 = c2[0]
            y2 = c2[1]
            try:
                if imageWithBlack[x2, y2][0] != 0 and imageWithBlack[x2, y2][1] != 0 and imageWithBlack[x2, y2][2] != 0:
                    r = r + img2[x2, y2][0]
                    g = g + img2[x2, y2][1]
                    b = b + img2[x2, y2][2]
                    count = count + 1
            except:
                exceptions = exceptions + 1
            if count > 0:
                img2[x,y] = [r/count, g/count, b/count]

    return img2

In [18]:
def resizeImage(src, scale_percent):

    #calculate the 50 percent of original dimensions
    width = int(src.shape[1] * scale_percent / 100)
    height = int(src.shape[0] * scale_percent / 100)

    # dsize
    dsize = (width, height)

    # resize image
    return cv2.resize(src, dsize)

In [19]:
def removeHairs(imageIndex):

    img =  cv2.cvtColor(cv2.imread(imageNames[imageIndex]), cv2.COLOR_BGR2RGB)
    img = resizeImage(img, 50)

    image = cv2.imread(imageNames[imageIndex],0) # load as 1-channel 8bit grayscale
    image = resizeImage(image, 50)

    edges = cv2.Canny(image,100,200)
#     print(imageIndex, "edges")
    
    edges2 =cv2.bilateralFilter(edges,7,300,9)
#     print(imageIndex, "edges2")
    
    edgeCoordinates = zipCoordinates(edges2)
#     plt.imshow(img)
    blackHighlights = getBlackHighlights(img, edgeCoordinates)
#     print(imageIndex, "blackHighlights")
    
    final = blurHairs2(img, edges2, blackHighlights)

    writeToFile(imageIndex,  cv2.cvtColor(final, cv2.COLOR_RGB2BGR))

In [9]:
import time
numberOfStartedImages = 20

threads = []
def startNewThreads(numberOfStartedImages):
    temp = numberOfStartedImages
    for t in threads:
        if t.isAlive() == False:
            t = threading.Thread(target = removeHairs, args = (temp, ))
            t.start()
            temp = temp + 1
#             if temp % 10 == 0:
#                 print("numberOfStartedImages", temp)
    return temp


for i in range(numberOfStartedImages):
    t = threading.Thread(target = removeHairs, args = (i, ))
    threads.append(t)
    t.start()
    time.sleep(0.5)

while numberOfStartedImages < 1202:
    temp = numberOfStartedImages
    numberOfStartedImages = startNewThreads(numberOfStartedImages)
    if numberOfStartedImages == temp:
        time.sleep(1)
        
for t in threads:
    t.join()
    
print('\nDone')


Processed: 539/1212 ==> 44.5%                                                                                                                 
Done


In [20]:
import time
numberOfStartedImages = 0
maxThreads = 6

threads = []
def startNewThreads(numberOfStartedImages):
    temp = numberOfStartedImages
    for t in threads:
        if t.isAlive() == False:
            t = threading.Thread(target = removeHairs, args = (temp, ))
            t.start()
            temp = temp + 1
#             if temp % 10 == 0:
#             global numberOfProcessedImages
#             print("numberOfStartedImages:", temp, "numberOdProcessedImages:", numberOfProcessedImages)
            if temp - numberOfStartedImages >= maxThreads:
                return temp
    return temp


for i in range(maxThreads):
    t = threading.Thread(target = removeHairs, args = (i, ))
    threads.append(t)
    t.start()
    numberOfStartedImages = numberOfStartedImages + 1
#     print("numberOfStartedImages:", numberOfStartedImages, "numberOdProcessedImages:", numberOfProcessedImages)
    time.sleep(1)

while numberOfStartedImages < 1202:
    temp = numberOfStartedImages
    numberOfStartedImages = startNewThreads(numberOfStartedImages)
    if numberOfStartedImages == temp:
        time.sleep(5)
    else:
        time.sleep(30)
        
for t in threads:
    t.join()
    
print('\nDone')

Processed: 1122/1210 ==> 92.7%                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
Done


In [21]:
threads = []

for i in range(6):
    t = threading.Thread(target = removeHairs, args = (1204+i, ))
    threads.append(t)
    t.start()
#     numberOfStartedImages = numberOfStartedImages + 1
#     print("numberOfStartedImages:", numberOfStartedImages, "numberOdProcessedImages:", numberOfProcessedImages)
    time.sleep(5)
    
for t in threads:
    t.join()
    
print('\nDone')

Processed: 1210/1210 ==> 100.0%                         
Done
